In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import sqlite3
import os

In [2]:
def generate_id(prefix, num):
    return f"{prefix}{num:03d}"

def random_string(length):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(random.choice(letters) for i in range(length))

def query_db(query, params=()):
    conn = sqlite3.connect('clash_royale.db')
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

In [3]:
descricoes_para_cartas = [
    "O Dragão Infernal queima tudo em seu caminho com um raio de fogo concentrado, deixando apenas cinzas.",
    "O Mago de Gelo congela seus inimigos com uma rajada de frio ártico, desacelerando seus movimentos.",
    "O Cavaleiro Fantasma emerge das sombras para causar dano letal antes de desaparecer sem deixar rastros.",
    "A Bruxa Sombria convoca morcegos das trevas enquanto lança feitiços devastadores contra seus oponentes.",
    "O Golem de Pedra avança lentamente, mas explode em mini-golems quando derrotado, continuando a batalha.",
    "A Princesa atira flechas flamejantes de longa distância, atingindo inimigos antes que eles a vejam.",
    "O Barril de Esqueletos lança um exército de esqueletos direto na linha de frente inimiga.",
    "O P.E.K.K.A. é uma máquina de combate blindada com uma espada gigante que corta qualquer coisa em seu caminho.",
    "O Gigante Real lança pedras enormes de sua besta, derrubando torres com facilidade.",
    "Os Três Mosqueteiros são uma força formidável, disparando uma chuva de balas que pode derrubar qualquer exército.",
    "O Lenhador enlouquecido derruba inimigos com seu machado, deixando uma poça de fúria ao ser derrotado.",
    "A Bandida desliza pelo campo de batalha, desferindo golpes críticos em suas vítimas antes de desaparecer.",
    "O Arqueiro Mágico dispara flechas que perfuram inimigos, atingindo vários alvos com um único tiro.",
    "O Mineiro cavouca subterraneamente para surgir onde menos se espera, causando dano diretamente às torres inimigas.",
    "O Mega Cavaleiro salta de inimigo em inimigo, esmagando-os com seu peso brutal.",
    "O Balão Esqueleto voa sobre as defesas, soltando bombas mortais diretamente nas torres adversárias.",
    "A Lava Hound libera uma enxurrada de lava-pups quando abatido, continuando o ataque aéreo.",
    "O Sparky carrega uma explosão elétrica devastadora, obliterando qualquer coisa que cruze seu caminho.",
    "Os Guardas Reais protegem suas torres com escudos robustos e lanças afiadas, enfrentando qualquer ameaça.",
    "O Mega Servo é uma criatura voadora blindada que ataca com garras afiadas, ideal para combates aéreos."
]

In [4]:
escala = 10
# Para cada grau desta escala são criados
# 4 players, 10 battles, 20 battleplayers, 20 decks, 160 deckcards
# Refiro-me, em cada um dos casos acima, a entidades distintas.
# O número de cartas distintas criadas é sempre 20

# Generate players
players = [
    {
        "playerTag": generate_id("PL", i),
        "name": f"Player{i}",
        "expLevel": random.randint(1, 13),
        "trophies": random.randint(0, 6000),
        "bestTrophies": random.randint(0, 6000),
        "wins": random.randint(0, 5000),
        "losses": random.randint(0, 5000),
        "battleCount": random.randint(0, 10000),
        "threeCrownWins": random.randint(0, 1000),
        "challengeCardsWon": random.randint(0, 1000),
        "challengeMaxWins": random.randint(0, 12),
        "tournamentCardsWon": random.randint(0, 1000),
        "tournamentBattleCount": random.randint(0, 1000),
        "role": random.choice(["member", "elder", "co-leader", "leader"]),
        "donations": random.randint(0, 5000),
        "donationsReceived": random.randint(0, 5000),
        "totalDonations": random.randint(0, 10000),
        "warDayWins": random.randint(0, 100),
        "clanCardsCollected": random.randint(0, 10000)
    }
    for i in range(1, escala * 4 + 1)
]

# Generate cards
cards = [
    {
        "cardId": generate_id("C", i),
        "name": f"Card{i}",
        "maxLevel": random.randint(1, 13),
        "iconUrl": f"http://example.com/card{i}.png",
        "rarity": random.choice(["common", "rare", "epic", "legendary"]),
        "description": descricoes_para_cartas[i-1]
    }
    for i in range(1, 21) # escala * 20 + 1
]

# Generate decks
decks = [
    {
        "deckId": generate_id("D", i)
    }
    for i in range(1, escala * 20 + 1)
]

# Generate deck cards ensuring each deck has 8 unique cards
deck_cards = []
for deck in decks:
#     card_ids = random.sample([card["cardId"] for card in cards], 8)
    card_ids = list(np.random.choice([card["cardId"] for card in cards], 8, replace=True))
    for card_id in card_ids:
        deck_cards.append({
            "deckCardId": generate_id("DC", len(deck_cards) + 1),
            "deckId": deck["deckId"],
            "cardId": card_id,
            "cardLevel": random.randint(1, 13)
        })

# Generate battles
# Não escolhe ainda o vencedor.
battles = [
    {
        "battleId": generate_id("B", i),
        "battleTime": datetime.now() - timedelta(days=random.randint(0, 365)),
        "gameMode": random.choice(["1v1", "2v2"]),
        "deckSelection": random.choice(["predefined", "custom"]),
#         "winner": random.choice([f"PL{j:03d}" for j in range(1, escala * 4 + 1)]),
        "trophyChange": random.randint(-30, 30),
        "crowns": random.randint(0, 3),
        "arena": random.choice(["Training Camp", "Goblin Stadium", "Bone Pit", "Barbarian Bowl"])
    }
    for i in range(1, escala * 10 + 1)
]

# Generate battle players ensuring each battle has 2 players
battle_players = []
for battle in battles:
    participantes = random.sample(players, 2)
    for i in range(2):
        player = participantes[i]
        deck = random.choice(decks)
        battle_players.append({
            "battlePlayerId": generate_id("BP", len(battle_players) + 1),
            "battleId": battle["battleId"],
            "playerTag": player["playerTag"],
            "startingTrophies": random.randint(0, 6000),
            "crownsEarned": random.randint(0, 3),
            "kingTowerHitPoints": random.randint(0, 5000),
            "princessTowerHitPoints": f"{random.randint(0, 5000)},{random.randint(0, 5000)}",
            "clanName": f"Clan{random.randint(1, 10)}",
            "clanTag": random_string(5),
            "deckId": deck["deckId"]
        })
    battle['winner'] = random.choice([battle_players[-1]['playerTag'], battle_players[-2]['playerTag']])

# Apenas para facilitar visualização dos dados sintéticos criados:
df_players = pd.DataFrame(players)
df_cards = pd.DataFrame(cards)
df_decks = pd.DataFrame(decks)
df_deck_cards = pd.DataFrame(deck_cards)
df_battles = pd.DataFrame(battles)
df_battle_players = pd.DataFrame(battle_players)

In [5]:
# Esta célula cria o arquivo clash_royale.db com os dados sintéticos criados

# Primeiramente, apaga o arquivo clash_royale.db, caso existente, para criar um novo
if os.path.exists('clash_royale.db'):
    # Apaga o arquivo
    os.remove('clash_royale.db')

# Cria a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Cria as tabelas
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    playerTag TEXT PRIMARY KEY,
    name TEXT,
    expLevel INTEGER,
    trophies INTEGER,
    bestTrophies INTEGER,
    wins INTEGER,
    losses INTEGER,
    battleCount INTEGER,
    threeCrownWins INTEGER,
    challengeCardsWon INTEGER,
    challengeMaxWins INTEGER,
    tournamentCardsWon INTEGER,
    tournamentBattleCount INTEGER,
    role TEXT,
    donations INTEGER,
    donationsReceived INTEGER,
    totalDonations INTEGER,
    warDayWins INTEGER,
    clanCardsCollected INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cards (
    cardId TEXT PRIMARY KEY,
    name TEXT,
    maxLevel INTEGER,
    iconUrl TEXT,
    rarity TEXT,
    description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS decks (
    deckId TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS deck_cards (
    deckCardId TEXT PRIMARY KEY,
    deckId TEXT,
    cardId TEXT,
    cardLevel INTEGER,
    FOREIGN KEY(deckId) REFERENCES decks(deckId),
    FOREIGN KEY(cardId) REFERENCES cards(cardId)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    battleId TEXT PRIMARY KEY,
    battleTime TEXT,
    gameMode TEXT,
    deckSelection TEXT,
    winner TEXT,
    trophyChange INTEGER,
    crowns INTEGER,
    arena TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battle_players (
    battlePlayerId TEXT PRIMARY KEY,
    battleId TEXT,
    playerTag TEXT,
    startingTrophies INTEGER,
    crownsEarned INTEGER,
    kingTowerHitPoints INTEGER,
    princessTowerHitPoints TEXT,
    clanName TEXT,
    clanTag TEXT,
    deckId TEXT,
    FOREIGN KEY(battleId) REFERENCES battles(battleId),
    FOREIGN KEY(playerTag) REFERENCES players(playerTag),
    FOREIGN KEY(deckId) REFERENCES decks(deckId)
)
''')

# Insere os dados nas tabelas
for player in players:
    cursor.execute('''
    INSERT INTO players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(player.values()))

for card in cards:
    cursor.execute('''
    INSERT INTO cards VALUES (?, ?, ?, ?, ?, ?)
    ''', tuple(card.values()))

for deck in decks:
    cursor.execute('''
    INSERT INTO decks VALUES (?)
    ''', (deck['deckId'],))

for deck_card in deck_cards:
    cursor.execute('''
    INSERT INTO deck_cards VALUES (?, ?, ?, ?)
    ''', tuple(deck_card.values()))

for battle in battles:
    cursor.execute('''
    INSERT INTO battles VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        battle['battleId'],
        battle['battleTime'].isoformat(),
        battle['gameMode'],
        battle['deckSelection'],
        battle['winner'],
        battle['trophyChange'],
        battle['crowns'],
        battle['arena']
    ))

for battle_player in battle_players:
    cursor.execute('''
    INSERT INTO battle_players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(battle_player.values()))

# Salva (commit) as mudanças e fecha a conexão com o banco de dados
conn.commit()
conn.close()


In [6]:
################################
####### TESTE DE QUERY 1 #######
################################

# Reabre a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Executa a consulta SQL
query = '''
SELECT *
FROM battle_players bp
INNER JOIN decks d ON bp.deckId = d.deckId;
'''
cursor.execute(query)

# Obtém os resultados da consulta
results = cursor.fetchall()

# Imprime os resultados
for row in results:
    print(row)

# Fecha a conexão com o banco de dados
conn.close()


('BP001', 'B001', 'PL012', 5744, 3, 4397, '3766,3723', 'Clan1', 'FRPEO', 'D103', 'D103')
('BP002', 'B001', 'PL035', 1136, 1, 1486, '4381,1463', 'Clan3', 'GDMSL', 'D013', 'D013')
('BP003', 'B002', 'PL007', 5162, 3, 2671, '2642,4595', 'Clan3', 'CKQNZ', 'D015', 'D015')
('BP004', 'B002', 'PL025', 2139, 0, 3342, '1265,3502', 'Clan2', 'PLMZO', 'D054', 'D054')
('BP005', 'B003', 'PL018', 4537, 0, 4348, '1296,2416', 'Clan7', 'LABLK', 'D115', 'D115')
('BP006', 'B003', 'PL039', 3050, 2, 1494, '519,1361', 'Clan5', 'DISBJ', 'D103', 'D103')
('BP007', 'B004', 'PL019', 2015, 2, 350, '468,4750', 'Clan5', 'WDLXQ', 'D087', 'D087')
('BP008', 'B004', 'PL023', 749, 0, 3786, '2025,2964', 'Clan9', 'JAZBT', 'D068', 'D068')
('BP009', 'B005', 'PL038', 2811, 0, 970, '3145,3436', 'Clan10', 'LPGCU', 'D091', 'D091')
('BP010', 'B005', 'PL007', 241, 3, 4864, '3291,4504', 'Clan3', 'RQSOE', 'D183', 'D183')
('BP011', 'B006', 'PL002', 3001, 3, 860, '3805,3498', 'Clan5', 'NDMHY', 'D139', 'D139')
('BP012', 'B006', 'PL006', 

In [7]:
################################
####### TESTE DE QUERY 2 #######
################################

# Função para obter deck_cards associados a um battle_player específico
def get_deck_cards_for_battle_player(battlePlayerId):
    # Reabre a conexão com o banco de dados
    conn = sqlite3.connect('clash_royale.db')
    cursor = conn.cursor()

    # Consulta SQL
    query = '''
    SELECT dc.*
    FROM battle_players bp
    INNER JOIN decks d ON bp.deckId = d.deckId
    INNER JOIN deck_cards dc ON d.deckId = dc.deckId
    WHERE bp.battlePlayerId = ?;
    '''

    # Executa a consulta com o battlePlayerId especificado
    cursor.execute(query, (battlePlayerId,))
    results = cursor.fetchall()

    # Fecha a conexão com o banco de dados
    conn.close()

    return results

# Exemplo de uso da função
battlePlayerId = 'BP001'  # Substitua pelo battlePlayerId desejado
deck_cards = get_deck_cards_for_battle_player(battlePlayerId)

# Imprime os resultados
for deck_card in deck_cards:
    print(deck_card)


('DC817', 'D103', 'C001', 3)
('DC818', 'D103', 'C006', 6)
('DC819', 'D103', 'C009', 13)
('DC820', 'D103', 'C013', 3)
('DC821', 'D103', 'C018', 9)
('DC822', 'D103', 'C002', 2)
('DC823', 'D103', 'C004', 9)
('DC824', 'D103', 'C011', 9)


<h3>Tarefa 1: Calcular a porcentagem de vitórias e derrotas utilizando a carta X em um intervalo de timestamps</h3>

In [8]:
def win_loss_percentage_with_card(card_id, start_time, end_time):
    query = '''
    SELECT bp.playerTag, b.winner
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ?
    '''
    df = query_db(query, (card_id, start_time, end_time))

    total_battles = len(df)
    if total_battles == 0:
        return {"win_percentage": 0, "loss_percentage": 0}

    wins = df[df['playerTag'] == df['winner']].shape[0]
    losses = total_battles - wins

    win_percentage = (wins / total_battles) * 100
    loss_percentage = (losses / total_battles) * 100

    return {"win_percentage": win_percentage, "loss_percentage": loss_percentage}

# Exemplo de uso:
card_id = 'C001'
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(win_loss_percentage_with_card(card_id, start_time, end_time))

{'win_percentage': 45.45454545454545, 'loss_percentage': 54.54545454545454}


<h3>Tarefa 2: Listar os decks completos que produziram mais de X% de vitórias em um intervalo de timestamps</h3>

In [9]:
def decks_with_high_win_rate(win_rate_threshold, start_time, end_time):
    query = '''
    SELECT bp.deckId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId
    HAVING (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, win_rate_threshold))
    return df

# Exemplo de uso:
win_rate_threshold = 50  # decks com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(decks_with_high_win_rate(win_rate_threshold, start_time, end_time))


   deckId  total_battles  wins
0    D007              2     2
1    D009              1     1
2    D012              2     2
3    D019              1     1
4    D020              1     1
5    D027              2     2
6    D035              1     1
7    D036              1     1
8    D042              1     1
9    D043              1     1
10   D044              2     2
11   D048              1     1
12   D051              1     1
13   D054              1     1
14   D061              1     1
15   D063              1     1
16   D064              1     1
17   D065              1     1
18   D066              1     1
19   D069              1     1
20   D071              2     2
21   D072              3     2
22   D073              4     3
23   D078              1     1
24   D079              1     1
25   D081              3     2
26   D084              1     1
27   D087              1     1
28   D089              1     1
29   D090              1     1
30   D091              1     1
31   D09

<h3>Tarefa 3: Calcular a quantidade de derrotas utilizando um combo de cartas em um intervalo de timestamps</h3>

In [10]:
def losses_with_card_combo(card_ids, start_time, end_time):
    card_placeholders = ', '.join('?' for _ in card_ids)
    query = f'''
    SELECT bp.battleId, COUNT(DISTINCT dc.cardId) as card_count
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId IN ({card_placeholders}) AND b.battleTime BETWEEN ? AND ?
    GROUP BY bp.battleId
    HAVING card_count = ?
    '''
    df = query_db(query, (*card_ids, start_time, end_time, len(card_ids)))

    total_battles = len(df)
    losses = total_battles - df[df['battleId'].isin(df['battleId'])].shape[0]

    return {"losses": losses}

# Exemplo de uso:
card_ids = ['C002', 'C003']
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(losses_with_card_combo(card_ids, start_time, end_time))


{'losses': 0}


<h3>Tarefa 3 versão 2 - utilizando apenas Python, sem SQL</h3>

In [11]:
derrotas_do_combo = 0
# vitorias_do_combo = 0
for batalha in df_battles[(df_battles['battleTime'] >= start_time) & (df_battles['battleTime'] <= end_time)]['battleId']:
    df_participantes = df_battle_players[df_battle_players['battleId']==batalha]
    vencedor = df_battles[df_battles['battleId']==batalha]['winner'].iloc[0]

    deque_perdedor = df_participantes.loc[df_participantes['playerTag'] != vencedor, 'deckId'].iloc[0]
#     deque_vencedor = df_participantes.loc[df_participantes['playerTag'] == vencedor, 'deckId'].iloc[0]
    cartas_perdedoras = df_deck_cards[df_deck_cards['deckId']==deque_perdedor]['cardId'].tolist()
#     cartas_vencedoras = df_deck_cards[df_deck_cards['deckId']==deque_vencedor]['cardId'].tolist()    
    if all([card in cartas_perdedoras for card in card_ids]):
        derrotas_do_combo+=1
#     if all([card in cartas_vencedoras for card in card_ids]):
#         vitorias_do_combo+=1
derrotas_do_combo

4

<h3>Tarefa 4: Calcular a quantidade de vitórias envolvendo a carta X onde o vencedor possui Z% menos troféus que o perdedor, a partida durou menos de 2 minutos, e o perdedor derrubou ao menos duas torres</h3>

In [12]:
def wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time):
    query = '''
    SELECT bp.battleId, bp.playerTag, bp.startingTrophies, b.winner, b.battleTime,
           (bp.kingTowerHitPoints + bp.princessTowerHitPoints) as totalHitPoints
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ? AND totalHitPoints < 2
    '''
    df = query_db(query, (card_id, start_time, end_time))
    
    # Filtrando as condições adicionais
    filtered_df = df[(df['winner'] == df['playerTag']) & 
                     ((df['startingTrophies'] * (1 - trophy_diff_percentage / 100)) > df['startingTrophies']) & 
                     (pd.to_datetime(df['battleTime']) - pd.to_datetime(df['battleTime']) < pd.Timedelta(minutes=2))]
    
    wins = filtered_df.shape[0]
    
    return {"wins": wins}

# Exemplo de uso:
card_id = 'C001'
trophy_diff_percentage = 20
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time))


{'wins': 0}


<h3>Tarefa 5: Listar o combo de cartas de tamanho N que produziram mais de Y% de vitórias em um intervalo de timestamps</h3>

In [13]:
def card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time):
    query = f'''
    SELECT bp.deckId, dc.cardId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId, dc.cardId
    HAVING COUNT(DISTINCT dc.cardId) = ? AND (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, combo_size, win_rate_threshold))
    return df

# Exemplo de uso:
combo_size = 2
win_rate_threshold = 50  # combos com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time))


Empty DataFrame
Columns: [deckId, cardId, total_battles, wins]
Index: []


<h3>Consultas adicionais I - Taxa de vitórias por raridade da carta</h3>
<p><i>Nesta consulta analisaremos o desempenho de cartas de diferentes raridades (comum, rara, épica, lendária) em todas as batalhas. A proposta é auxiliar a identificar se cartas de raridades específicas apresentam desempenho superior ou inferior.</i></p>

In [14]:
def win_rate_by_rarity():
    query = '''
    SELECT c.rarity, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.rarity
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['total_battles']) * 100
    return df

# Exemplo de uso:
print(win_rate_by_rarity())


      rarity  total_battles  wins   win_rate
0     common            279   141  50.537634
1       epic            275   138  50.181818
2  legendary            699   339  48.497854
3       rare            347   182  52.449568


<h3>Consultas adicionais II - Análise de diversidade de decks</h3>
<p><i>Nesta consulta examinaremos a variedade dos decks utilizados em partidas com alta taxa de troféus. A baixa diversidade de cartas no deck pode ser um indício de que certas combinações se tornam muito poderosas e precisam de equilíbrio.</i></p>

In [15]:
def deck_diversity(high_trophy_threshold):
    query = '''
    SELECT bp.deckId, COUNT(DISTINCT dc.cardId) as unique_cards
    FROM battle_players bp
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    INNER JOIN players p ON bp.playerTag = p.playerTag
    WHERE p.trophies > ?
    GROUP BY bp.deckId
    '''
    df = query_db(query, (high_trophy_threshold,))
    
    return df

# Exemplo de uso:
high_trophy_threshold = 5000  # Ajuste este valor conforme necessário
print(deck_diversity(high_trophy_threshold))


  deckId  unique_cards
0   D002             7
1   D007             5
2   D088             6
3   D101             7
4   D103             8
5   D178             7
6   D182             7
7   D189             7


<h3>Consultas adicionais III - Taxa de uso do card vs taxa de vitórias</h3>
<p><i>Nesta consulta compararemos a frequência de uso de um card com sua taxa de ganho. Cards com altas taxas de utilização mas com baixas taxas de ganho podem ser consideradas mais fortes do que realmente são, enquanto cards com baixas taxas de utilização mas altas taxas de vitórias podem ser subvalorizados. </i></p>

In [16]:
def card_usage_vs_win_rate():
    query = '''
    SELECT c.cardId, c.name, c.rarity,
           COUNT(*) as usage_count,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.cardId
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['usage_count']) * 100
    return df

# Exemplo de uso:
print(card_usage_vs_win_rate())


   cardId    name     rarity  usage_count  wins   win_rate
0    C001   Card1  legendary           77    35  45.454545
1    C002   Card2       rare           75    42  56.000000
2    C003   Card3       epic           68    33  48.529412
3    C004   Card4     common           82    47  57.317073
4    C005   Card5       epic           69    37  53.623188
5    C006   Card6     common           75    40  53.333333
6    C007   Card7  legendary           74    35  47.297297
7    C008   Card8  legendary           92    39  42.391304
8    C009   Card9  legendary           90    47  52.222222
9    C010  Card10       rare           95    48  50.526316
10   C011  Card11  legendary          101    53  52.475248
11   C012  Card12       rare           78    43  55.128205
12   C013  Card13  legendary           98    41  41.836735
13   C014  Card14     common           61    24  39.344262
14   C015  Card15  legendary           72    39  54.166667
15   C016  Card16       epic           70    32  45.7142